# Different ways to combine a 2 microscopy image with a corresponding labels image into an ome-zarr file

## 1. Microscopy image and labels represent two regular channels in ome-zarr

In [12]:
import os
import numpy as np
import zarr

from tifffile import imread
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image, add_metadata


# --------------------------------------------------------
# Input TIFF paths (adjust as needed)
# --------------------------------------------------------
tiff_original = "denbi/sample.tiff"
tiff_labels   = "denbi/labels.tiff"

# --------------------------------------------------------
# Read TIFFs
# --------------------------------------------------------
img_original = imread(tiff_original)
img_labels   = imread(tiff_labels)

# Ensure at least 3D (Z, Y, X)
if img_original.ndim == 2:
    img_original = img_original[np.newaxis, :, :]   # (1, Y, X)
if img_labels.ndim == 2:
    img_labels = img_labels[np.newaxis, :, :]

# Check shapes are compatible
if img_original.shape != img_labels.shape:
    raise ValueError(
        f"Shape mismatch: original {img_original.shape} vs labels {img_labels.shape}"
    )

# Cast to 16-bit
img_original = img_original.astype(np.uint16)
img_labels   = img_labels.astype(np.uint16)

# Stack as channels → (C=2, Z, Y, X)
data = np.stack([img_original, img_labels], axis=0)

print("Final data shape (C, Z, Y, X):", data.shape)

# --------------------------------------------------------
# Output Zarr path
# --------------------------------------------------------
path = "test-zarr/2ch.ome.zarr"
os.makedirs(path, exist_ok=True)

store = parse_url(path, mode="w").store
root = zarr.group(store=store)

# --------------------------------------------------------
# Write image (OME-NGFF v0.4 style, zarr v2)
# Axes: C, Z, Y, X
# --------------------------------------------------------
chunks = (1, 1, data.shape[-2], data.shape[-1])  # chunk per (C,Z), full XY

write_image(
    image=data,
    group=root,
    axes="czyx",
    storage_options=dict(chunks=chunks),
)

# --------------------------------------------------------
# Add OMERO-style metadata with channel labels
# --------------------------------------------------------
max_val = int(data.max()) if data.size > 0 else 65535

add_metadata(root, {
    "omero": {
        "channels": [
            {
                "color": "FFFFFF",  # white
                "window": {"start": 0, "end": max_val},
                "label": "original",
                "active": True,
            },
            {
                "color": "FF00FF",  # magenta
                "window": {"start": 0, "end": max_val},
                "label": "labels",
                "active": True,
            },
        ]
    }
})

print(f"Written OME-Zarr to: {os.path.abspath(path)}")


Final data shape (C, Z, Y, X): (2, 1, 520, 704)
Written OME-Zarr to: /home/peter/sciebo/denbi25/omero-roi-annotations/test-zarr/2ch.ome.zarr


## 2. Microscopy image is a regular channel, but labels are added as labels explicitly in ome-zarr


In [13]:
import os
import numpy as np
import zarr
import tifffile

from ome_zarr.io import parse_url
from ome_zarr.writer import write_image, add_metadata

# --------------------------------------------------------
# Input TIFF paths
# --------------------------------------------------------
original_path = "denbi/sample.tiff"
labels_path   = "denbi/labels.tiff"

# --------------------------------------------------------
# Read TIFFs
# --------------------------------------------------------
orig = tifffile.imread(original_path)
lab  = tifffile.imread(labels_path)

# Basic shape checks
if orig.shape != lab.shape:
    raise ValueError(f"Shape mismatch: original {orig.shape} vs labels {lab.shape}")

# Decide axes string based on dimensionality
# (adapt if you have T/C in your TIFFs)
if orig.ndim == 2:
    axes = "yx"
elif orig.ndim == 3:
    axes = "zyx"
else:
    raise ValueError(
        f"Unsupported number of dimensions: {orig.ndim}. "
        "This example assumes 2D (Y,X) or 3D (Z,Y,X) arrays."
    )

# Cast types (optional but typical)
orig = orig.astype(np.uint16)
# labels are usually integer-labeled regions
lab  = lab.astype(np.int32)

# --------------------------------------------------------
# Create OME-Zarr root
# --------------------------------------------------------
zarr_path = "test-zarr/1ch_plus_explicit_labels.ome.zarr"
os.makedirs(zarr_path, exist_ok=True)

store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# --------------------------------------------------------
# 1) Write the original image to the ROOT group
# --------------------------------------------------------
# chunks: one can tune this; here we use full XY, chunk in Z if 3D
if axes == "yx":
    chunks = (256, 256)                     # Y,X
else:  # "zyx"
    chunks = (1, orig.shape[-2], orig.shape[-1])  # Z,Y,X

write_image(
    image=orig,
    group=root,
    axes=axes,
    storage_options=dict(chunks=chunks),
)

# Optional: OMERO rendering metadata for the original
min_val = int(orig.min())
max_val = int(orig.max()) if orig.size > 0 else 65535

add_metadata(root, {
    "omero": {
        "channels": [{
            "color": "FFFFFF",  # white
            "window": {"start": min_val, "end": max_val,
                       "min": min_val, "max": max_val},
            "label": "original",
            "active": True,
        }]
    }
})

# --------------------------------------------------------
# 2) Write the labels image under /labels as a dedicated label
# --------------------------------------------------------

# /labels group
labels_grp = root.create_group("labels")

# name of this specific label image
label_name = "labels"

# The 'labels' attribute on /labels lists all label datasets
add_metadata(labels_grp, {"labels": [label_name]})

# group for this specific label image: /labels/labels
label_grp = labels_grp.create_group(label_name)

# write label array itself
write_image(
    image=lab,
    group=label_grp,
    axes=axes,
    storage_options=dict(chunks=chunks),
)

# image-label metadata: tells tools this is a label image and
# can also define colors for label values
#
# For simplicity we just define a couple of example label values.
# You can adapt this to your actual segmentation labels.
# --------------------------------------------------------
# Auto-generate colors for all label values
# --------------------------------------------------------

import random

# Find unique label IDs (e.g. [0, 1, 2, 5, 37, ...])
unique_labels = np.unique(lab)

colors_meta = []

for label_value in unique_labels:
    if label_value == 0:
        # Background (0) → transparent or fixed color
        colors_meta.append({
            "label-value": int(label_value),
            "rgba": [0, 0, 0, 0]   # fully transparent
        })
    else:
        # Random bright color for each label
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)

        colors_meta.append({
            "label-value": int(label_value),
            "rgba": [r, g, b, 255]     # opaque
        })

# Attach metadata to the label group
add_metadata(label_grp, {
    "image-label": {
        "colors": colors_meta
    }
})



print(f"Wrote OME-Zarr with labels to: {os.path.abspath(zarr_path)}")


Wrote OME-Zarr with labels to: /home/peter/sciebo/denbi25/omero-roi-annotations/test-zarr/1ch_plus_explicit_labels.ome.zarr


## 3. Mock up of the collection with nodes as proposed in RFC-8 on top of the previously created ome-zarr with the microscopy image being complementd with explicit labels

In [14]:
import os
import random
import numpy as np
import zarr
import tifffile

from ome_zarr.io import parse_url
from ome_zarr.writer import write_image, add_metadata


# --------------------------------------------------------
# Read TIFFs
# --------------------------------------------------------
original_path = "denbi/sample.tiff"
labels_path   = "denbi/labels.tiff"

# --------------------------------------------------------
# Read TIFFs
# --------------------------------------------------------
orig = tifffile.imread(original_path)
lab  = tifffile.imread(labels_path)

# Basic shape checks
if orig.shape != lab.shape:
    raise ValueError(f"Shape mismatch: original {orig.shape} vs labels {lab.shape}")

# Decide axes string based on dimensionality
# (adapt if you have T/C in your TIFFs)
if orig.ndim == 2:
    axes = "yx"
elif orig.ndim == 3:
    axes = "zyx"
else:
    raise ValueError(
        f"Unsupported number of dimensions: {orig.ndim}. "
        "This example assumes 2D (Y,X) or 3D (Z,Y,X) arrays."
    )

# Cast types (typical choices)
orig = orig.astype(np.uint16)   # intensities
lab  = lab.astype(np.int32)     # label IDs


# --------------------------------------------------------
# Create OME-Zarr root (collection)
# --------------------------------------------------------
zarr_path = "test-zarr/collection_combined_with_1ch_plus_labels.ome.zarr"
if os.path.exists(zarr_path):
    raise RuntimeError(f"{zarr_path} already exists, remove it first")

store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# --------------------------------------------------------
# 1) Write the ORIGINAL image as a multiscale under /original
# --------------------------------------------------------
orig_group = root.create_group("original")

# Choose chunks (you can tweak these)
if axes == "yx":
    chunks = (256, 256)  # Y,X
else:  # "zyx"
    chunks = (1, orig.shape[-2], orig.shape[-1])  # Z,Y,X

write_image(
    image=orig,
    group=orig_group,
    axes=axes,
    storage_options=dict(chunks=chunks),
)

# Add OMERO-style rendering metadata for the original
min_val = int(orig.min())
max_val = int(orig.max()) if orig.size > 0 else 65535

add_metadata(orig_group, {
    "omero": {
        "channels": [{
            "color": "FFFFFF",  # white
            "window": {"start": min_val, "end": max_val,
                       "min": min_val, "max": max_val},
            "label": "original",
            "active": True,
        }]
    }
})

# (Optional) You can also tag this group as a "multiscale" in RFC-8 style:
# orig_group.attrs["ome"] = {
#     "type": "multiscale",
#     "name": "original"
# }


# --------------------------------------------------------
# 2) Write the LABEL image using the labels extension
#    under /original/labels/labels
# --------------------------------------------------------
labels_root = orig_group.create_group("labels")
label_name = "labels"

# List the label datasets under /original/labels
add_metadata(labels_root, {"labels": [label_name]})

label_grp = labels_root.create_group(label_name)

write_image(
    image=lab,
    group=label_grp,
    axes=axes,
    storage_options=dict(chunks=chunks),
)

# Auto-generate colors for all label values
random.seed(12345)  # for reproducibility
unique_labels = np.unique(lab)

colors_meta = []
for label_value in unique_labels:
    label_value = int(label_value)
    if label_value == 0:
        # Background → transparent
        colors_meta.append({
            "label-value": label_value,
            "rgba": [0, 0, 0, 0],
        })
    else:
        # Random bright-ish color
        r = random.randint(80, 255)
        g = random.randint(80, 255)
        b = random.randint(80, 255)
        colors_meta.append({
            "label-value": label_value,
            "rgba": [r, g, b, 255],
        })

add_metadata(label_grp, {
    "image-label": {
        "colors": colors_meta
    }
})

# (Optional) RFC-8 tag for this group as a multiscale:
# label_grp.attrs["ome"] = {
#     "type": "multiscale",
#     "name": "labels"
# }


# --------------------------------------------------------
# 3) Add RFC-8 style COLLECTION metadata at the ROOT
# --------------------------------------------------------
dataset_name = os.path.splitext(os.path.basename(original_path))[0]

root.attrs["ome"] = {
    "version": "0.x",              # or whatever draft version you want to record
    "type": "collection",
    "name": dataset_name,          # e.g. "original"
    "nodes": [
        {
            "name": "original",
            "type": "multiscale",
            "path": "./original",
            "attributes": {
                # arbitrary viewer / user metadata
                "ome-iviewer:settings": {
                    "isDisabled": False
                },
                "ome-iviewer:voxelType": "intensity"
            },
        },
        {
            "name": "labels",
            "type": "multiscale",
            # Path to the label *image* (the multiscale array)
            "path": "./original/labels/labels",
            "attributes": {
                "ome-iviewer:voxelType": "labels"
            },
        },
    ],
    # Collection-level attributes (optional, add whatever you need)
    "attributes": {
        # You can put dataset-wide metadata here
        # "project": "my-awesome-project",
        # "sample_id": "1234",
    },
}

print(f"Wrote RFC-8-style collection to: {os.path.abspath(zarr_path)}")


Wrote RFC-8-style collection to: /home/peter/sciebo/denbi25/omero-roi-annotations/test-zarr/collection_combined_with_1ch_plus_labels.ome.zarr


## 4. Mock up of the collection with nodes as proposed in RFC-8 on top, but as in the beginning adding the intensity and the label image as equally ranked multiscale nodes specifying their respective nature as attributes

In [15]:
import os
import numpy as np
import zarr
import tifffile

from ome_zarr.io import parse_url
from ome_zarr.writer import write_image, add_metadata


# --------------------------------------------------------
# Read TIFFs
# --------------------------------------------------------
original_path = "denbi/sample.tiff"
labels_path   = "denbi/labels.tiff"

orig = tifffile.imread(original_path)
lab  = tifffile.imread(labels_path)

# Basic shape checks
if orig.shape != lab.shape:
    raise ValueError(f"Shape mismatch: original {orig.shape} vs labels {lab.shape}")

# Decide axes string based on dimensionality
# (adapt if you have T/C in your TIFFs)
if orig.ndim == 2:
    axes = "yx"
elif orig.ndim == 3:
    axes = "zyx"
else:
    raise ValueError(
        f"Unsupported number of dimensions: {orig.ndim}. "
        "This example assumes 2D (Y,X) or 3D (Z,Y,X) arrays."
    )

# Cast types (typical choices)
orig = orig.astype(np.uint16)   # intensities
lab  = lab.astype(np.int32)     # label-like data, but now stored as a normal image


# --------------------------------------------------------
# Create OME-Zarr root (collection)
# --------------------------------------------------------
zarr_path = "test-zarr/collection_with_multiscale_nodes_annotated_by_attributes.ome.zarr"
if os.path.exists(zarr_path):
    raise RuntimeError(f"{zarr_path} already exists, remove it first")

store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# --------------------------------------------------------
# 1) Write the ORIGINAL image as a multiscale under /original
# --------------------------------------------------------
orig_group = root.create_group("original")

# Choose chunks (you can tweak these)
if axes == "yx":
    chunks = (256, 256)  # Y,X
else:  # "zyx"
    chunks = (1, orig.shape[-2], orig.shape[-1])  # Z,Y,X

write_image(
    image=orig,
    group=orig_group,
    axes=axes,
    storage_options=dict(chunks=chunks),
)

# Add OMERO-style rendering metadata for the original
min_val = int(orig.min())
max_val = int(orig.max()) if orig.size > 0 else 65535

add_metadata(orig_group, {
    "omero": {
        "channels": [{
            "color": "FFFFFF",  # white
            "window": {"start": min_val, "end": max_val,
                       "min": min_val, "max": max_val},
            "label": "original",
            "active": True,
        }]
    }
})

# Optional: tag as multiscale in RFC-8 style
# orig_group.attrs["ome"] = {
#     "type": "multiscale",
#     "name": "original"
# }


# --------------------------------------------------------
# 2) Write the LABEL image as a *second* multiscale under /labels
#    (no labels extension, just another image)
# --------------------------------------------------------
labels_group = root.create_group("labels")

write_image(
    image=lab,
    group=labels_group,
    axes=axes,
    storage_options=dict(chunks=chunks),
)

# Optional OMERO metadata for the labels image (treated as another grayscale image)
lab_min = int(lab.min())
lab_max = int(lab.max()) if lab.size > 0 else 1

add_metadata(labels_group, {
    "omero": {
        "channels": [{
            "color": "FF00FF",  # magenta, arbitrary
            "window": {"start": lab_min, "end": lab_max,
                       "min": lab_min, "max": lab_max},
            "label": "labels",
            "active": True,
        }]
    }
})

# Optional: tag as multiscale in RFC-8 style
# labels_group.attrs["ome"] = {
#     "type": "multiscale",
#     "name": "labels"
# }


# --------------------------------------------------------
# 3) Add RFC-8 style COLLECTION metadata at the ROOT
# --------------------------------------------------------
dataset_name = os.path.splitext(os.path.basename(original_path))[0]

root.attrs["ome"] = {
    "version": "0.x",              # or whatever draft version you want to record
    "type": "collection",
    "name": dataset_name,          # e.g. derived from original filename
    "nodes": [
        {
            "name": "original",
            "type": "multiscale",
            "path": "./original",
            "attributes": {
                # arbitrary viewer / user metadata
                "ome-iviewer:settings": {
                    "isDisabled": False
                },
                "ome-iviewer:voxelType": "intensity"
            },
        },
        {
            "name": "labels",
            "type": "multiscale",
            # Path to the second multiscale image (now a regular node)
            "path": "./labels",
            "attributes": {
                "ome-iviewer:voxelType": "labels"
            },
        },
    ],
    # Collection-level attributes (optional, add whatever you need)
    "attributes": {
        # e.g. project, sample metadata, etc.
        # "project": "my-awesome-project",
        # "sample_id": "1234",
    },
}

print(f"Wrote RFC-8-style collection to: {os.path.abspath(zarr_path)}")


Wrote RFC-8-style collection to: /home/peter/sciebo/denbi25/omero-roi-annotations/test-zarr/collection_with_multiscale_nodes_annotated_by_attributes.ome.zarr
